In [71]:
import pandas as pd
import numpy as np
import random
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


In [72]:
df = pd.read_csv('2918001_JEQUIE.csv', sep=';')

C:\Users\guiwa\AppData\Local\Temp\ipykernel_8220\6535077.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2918001_JEQUIE.csv', sep=';')


In [82]:
df_aux = df[['NOM_TIPO_SEGLOGR', 'NOM_SEGLOGR', 'NUM_ENDERECO']]

In [86]:
df_aux = df_aux.sample(n=1000, random_state=42)

In [74]:
tipo_endereco = df_aux['NOM_TIPO_SEGLOGR'].iloc[0]
nome_endereco = df_aux['NOM_SEGLOGR'].iloc[0]
numero_endereco = df_aux['NUM_ENDERECO'].iloc[0]

endereco_completo = f"{tipo_endereco} {nome_endereco} {numero_endereco}"
endereco_completo

'RUA WILTON PIMENTA 28'

In [75]:
def introduzir_erros_df_aux(df, proporcao_erros=1.0):
    df_corrupto = df.copy()
    num_erros = int(len(df) * proporcao_erros)

    for _ in range(num_erros):
        idx = random.randint(0, len(df) - 1)
        erro_tipo = random.randint(1, 5)

        if erro_tipo == 1:
            df_corrupto.iloc[idx, df_corrupto.columns.get_loc("NOM_SEGLOGR")] = " ".join(
                str(df.iloc[idx]["NOM_SEGLOGR"]).split()[:-1])
        elif erro_tipo == 2:
            df_corrupto.iloc[idx, df_corrupto.columns.get_loc("NOM_TIPO_SEGLOGR")] = np.nan
        elif erro_tipo == 3:
            valor = str(df.iloc[idx]["NOM_SEGLOGR"])
            if len(valor) > 0:
                df_corrupto.iloc[idx, df_corrupto.columns.get_loc("NOM_SEGLOGR")] = \
                    valor[0].replace("W", "V") + valor[1:]
        elif erro_tipo == 4:
            df_corrupto.iloc[idx, df_corrupto.columns.get_loc("NUM_ENDERECO")] = np.nan
        elif erro_tipo == 5:
            valor = df.iloc[idx]["NUM_ENDERECO"]
            if not pd.isnull(valor):
                df_corrupto.iloc[idx, df_corrupto.columns.get_loc("NUM_ENDERECO")] = valor - 1

    return df_corrupto


In [ ]:
df_corrupto = introduzir_erros_df_aux(df_aux)
df_corrupto

,NOM_TIPO_SEGLOGR,NOM_SEGLOGR,NUM_ENDERECO
42199,RUA,WILTON,28


In [77]:
df_corrupto['NOM_SEGLOGR'] = 'UILTON PIMENTA'
df_corrupto['NOM_TIPO_SEGLOGR'] = 'RUA'

In [78]:
tipo_endereco_corrupto = df_corrupto['NOM_TIPO_SEGLOGR'].iloc[0]
nome_endereco_corrupto = df_corrupto['NOM_SEGLOGR'].iloc[0]
numero_endereco_corrupto = df_corrupto['NUM_ENDERECO'].iloc[0]

endereco_completo_corrupto = f"{tipo_endereco_corrupto} {nome_endereco_corrupto} {numero_endereco_corrupto}"
endereco_completo_corrupto

'RUA UILTON PIMENTA 28'

In [79]:

# 📌 Geração de embeddings com BERT

df_novo = pd.DataFrame()

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

embedding_normal = get_bert_embedding(endereco_completo)
embedding_corrupto = get_bert_embedding(endereco_completo_corrupto)

In [80]:
distancia = np.linalg.norm(embedding_normal - embedding_corrupto)
print(f"Distância Euclidiana: {distancia}")


Distância Euclidiana: 3.547633171081543


In [ ]:

# Aplicando PCA
scaler = StandardScaler()
X = np.stack(df["embedding_nome"].values)
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=4)
df["pca_embedding"] = list(pca.fit_transform(X_scaled))

KeyError: 'embedding_nome'